# Social infrastructure in LAC

**Objetive:** <br>
The following notebook runs all the social infrastructure analysis. <br>

Author: Laura Goyeneche, Consultant SPH, lauragoy@iadb.org <br>
Created: March 20, 2023

## 1. Basics

In [ ]:
%%capture
!pip install h3

In [ ]:
%%capture
# Import modules 
import fiona 
from utils import * 
from h3 import geo_to_h3, h3_to_geo_boundary
from shapely.geometry import Polygon
from zipfile import ZipFile

## 2. Inputs

In [ ]:
# Determine input values
data   = get_iadb()
codes  = data.isoalpha3.tolist()
groups = ["total_population","women","men","children_under_five","youth_15_24","elderly_60_plus","women_of_reproductive_age_15_49"]

In [ ]:
# Shapefiles 
shp1 = get_country_shp(level = 1)
shp2 = get_country_shp(level = 2)

## 3. Population

In [ ]:
%%capture
# Inputs
group = "total_population"
code  = "BRA"

# Get population data 
file = get_population(data, code, group)

# Save data 
file.to_csv(f"../data/0-raw/population/{group}/{code}_{group}.csv.gz", compression = 'gzip')

In [ ]:
# Calculate total population by adminsitrative level
group        = "total_population"
pop_shp1_lac = []
pop_shp2_lac = []

for code in codes[:-1]: 
    print(code)
    # Population Meta-level
    population = pd.read_csv(f"../data/0-raw/population/{group}/{code}_{group}.csv.gz")
    geometry   = gpd.points_from_xy(population['longitude'], population['latitude'])
    population = gpd.GeoDataFrame(population.copy(), geometry = geometry, crs = 4326)

    # Population in admin level 1
    pop_shp1 = gpd.sjoin(shp1[shp1.ADM0_PCODE == code], population)
    pop_shp1 = pop_shp1[["ADM1_PCODE","population"]]
    pop_shp1 = pop_shp1.groupby("ADM1_PCODE").sum().reset_index()
    
    # Population in admin level 2
    pop_shp2 = gpd.sjoin(shp2[shp2.ADM0_PCODE == code], population)
    pop_shp2 = pop_shp2[["ADM2_PCODE","population"]]
    pop_shp2 = pop_shp2.groupby("ADM2_PCODE").sum().reset_index()
    
    # Append to master lists
    pop_shp1_lac.append(pop_shp1)
    pop_shp2_lac.append(pop_shp2)

pop_shp1_lac = pd.concat(pop_shp1_lac)
pop_shp2_lac = pd.concat(pop_shp2_lac)

In [ ]:
# Export population datasets
    # Admin 1
shp1_ = shp1.drop(columns = "geometry")
shp1_ = shp1_.merge(pop_shp1_lac, on = "ADM1_PCODE", how = "left")
shp1_.to_csv(f"../data/0-raw/population/{group}/population-adm1.csv", index = False)

    # Admin 2
shp2_ = shp2.drop(columns = "geometry")
shp2_ = shp2_.merge(pop_shp2_lac, on = "ADM2_PCODE", how = "left")
shp2_.to_csv(f"../data/0-raw/population/{group}/population-adm2.csv", index = False)

In [ ]:
# DON'T RUN
# Datafile save in wrong format 
%%capture
for group in groups:
    for code in codes[:1]:
        data = f"../data/0-raw/population/{group}/{code}_{group}.csv.gz"
        data = pd.read_csv(data)
        path = "Development Data Partnership/Facebook - High resolution population density map/public-fb-data/csv/"
        path = scldatalake + f"{path}/{code.upper()}/{code}_{group}.csv.gz"
        data.to_csv(path, compression = 'gzip')

## 4. Amenities

In [ ]:
%%capture 
amenities = ['healthcare','financial']
groups    = ["official","public"]
for group in groups:
    for amenity in amenities:
        # Get infrastructure data
        infrastructure = get_amenity(amenity, group)
        
        if len(infrastructure) > 0:
            infrastructure.to_csv(f"../data/0-raw/infrastructure/{amenity}_facilities_{group}.csv")

## 4. Isochrones

### 4.1 Healthcare

In [ ]:
%%time 
# Inputs
amenity = 'healthcare'
minute  = 30
profile = 'driving'
group   = "official"
code    = "BRA"

# Get isochrones per amenity
isochrone = get_isochrones_country(code, amenity, minute, profile, group)
isochrone['geometry'] = isochrone.geometry.buffer(0)
isochrones = isochrone.dissolve()

# Export isochrone
isochrones.to_file(f"../data/1-isochrones/{amenity}/{group}/{minute}-min/{code}-{profile}-{minute}.geojson", driver = "GeoJSON")

### 4.2 Financial

In [ ]:
%%capture 
# Inputs
amenity = 'financial'
group   = "public"
minute  = 30
profile = 'driving'
code    = "GUY"

# Get isochrones per amenity
isochrone  = get_isochrones_country(code, amenity, minute, profile, group)
isochrones = isochrone.dissolve()

# Export isochrone 
isochrones.to_file(f"../data/1-isochrones/{amenity}/{code}-{profile}-{minute}.geojson", driver = "GeoJSON")

In [ ]:
# For each financial service type
for category in isochrone.amenity.unique():
    # Create one multipolygon
    isochrones = isochrone[isochrone.amenity == category]
    isochrones = isochrones.dissolve()

    # Export isochrone 
    isochrones.to_file(f"../data/1-isochrones/{amenity}/{category}/{code}-{profile}-{minute}.geojson", driver = "GeoJSON")

### 4.3 Regional shapefiles

#### 4.3.1 Individual regional shapefiles

In [ ]:
# Inputs
amenity    = "healthcare"
profile    = "driving"
minute     = 30
group      = "official"
isochrones = []

In [ ]:
# Create list of country-isochrones
# Note: Try/except for countries without isochrones
for code in codes:
    try:
        with fiona.Env(OGR_GEOJSON_MAX_OBJ_SIZE = 2000):  
            isochrone = gpd.read_file(f"../data/1-isochrones/{amenity}/{group}/{minute}-min/{code}-{profile}-{minute}.geojson")
            isochrones.append(isochrone)
    except:
        continue

In [ ]:
%%time 
# Create regional isochrone
isochrones = pd.concat(isochrones)
isochrones = isochrones.dissolve()

In [ ]:
%%capture
# Export isochrones 
isochrones.to_file(f"../data/1-isochrones/region/{amenity}/{group}/{minute}-min/{amenity}-{profile}-{minute}.shp")

In [ ]:
fig, ax = plt.subplots(1, figsize = (20,8))
shp1.plot(ax = ax, color = "#D6D6D6")
isochrones.plot(ax = ax)

#### 4.3.2 Joint regional shapefiles

In [ ]:
# Inputs 
amenity = "healthcare"
profile = "driving"
region_ = []

In [ ]:
# Create joint shapefile
minutes = [15, 30, 45]
for minute in minutes:
    temp = gpd.read_file(f"../data/1-isochrones/region/{amenity}-{profile}-{minute}.shp")
    temp["minute"] = minute
    region_.append(temp)

In [ ]:
# Create regional isochrone
region_ = pd.concat(region_)

In [ ]:
%%capture
# Export isochrones 
region_.to_file(f"../data/1-isochrones/{amenity}-{profile}.shp")

## 5. Coverage

### 5.1. Healthcare

In [ ]:
%%time 
# Inputs
amenity = 'healthcare'
minute  = 30
profile = 'driving'
group   = "official"
code    = "BRA"

# Create shapefile
adm2_, h3_ = get_coverage(code, amenity, profile, minute, group)

# Export isochrone 
adm2_.to_file(f"../data/2-coverage/{amenity}/adm2/{group}/{minute}-min/{code}-{profile}-{minute}.geojson", driver = "GeoJSON")
h3_  .to_file(f"../data/2-coverage/{amenity}/h3/{group}/{minute}-min/{code}-{profile}-{minute}.geojson"  , driver = "GeoJSON")

### 5.2. Financieros

In [ ]:
# Inputs
amenity    = 'financial'
subamenity = {"all":["","total"],"atm":["atm","atm"],"bank":["bank","bank"],"bureau_de_change":["bureau_de_change","bureau_de_change"]}
minute     = 30
profile    = 'driving'
code       = "BRA"

#for amenity_ in subamenity.keys():
for amenity_ in ["bank","bureau_de_change"]:
    # Labels 
    amenity_input  = f"{amenity}/{subamenity[amenity_][0]}"
    amenity_input  = re.sub("/$","",amenity_input)
    amenity_output = f"{amenity}/{subamenity[amenity_][1]}"

    # Create shapefile
    adm2_, h3_ = get_coverage(code, amenity_input, profile, minute)

    # Export isochrone
    adm2_.to_file(f"../data/2-coverage/{amenity_output}/adm2/{code}-{profile}-{minute}.geojson", driver = "GeoJSON")
    h3_  .to_file(f"../data/2-coverage/{amenity_output}/h3/{code}-{profile}-{minute}.geojson"  , driver = "GeoJSON")

### 5.3. Create regional shapefiles

In [ ]:
# Shapefile ADMIN-2
file = f"Geospatial Basemaps/Cartographic Boundary Files/LAC-26/region/level-2/lac-level-2.shp"
path = scldatalake + file
shp  = gpd.read_file(path)

#### 5.3.1. Healthcare amenities

In [ ]:
%%time 
# Create public and official regional shp
# Inputs 
unit  = "h3"
group = "official"
min_  = 30

# Append datasets
path  = f"../data/2-coverage/healthcare/{unit}/{group}/{min_}-min"
file  = os.listdir(path)
file  = [name for name in file if "geojson" in name]
shp_  = [gpd.read_file(f"{path}/{name}") for name in file]
lac   = gpd.pd.concat(shp_).pipe(gpd.GeoDataFrame)
lac.to_file(f"../data/2-coverage/healthcare/{unit}/{group}/lac-driving-{min_}-min.geojson", driver = "GeoJSON")

In [ ]:
%%time 
# Join public and official amenities
unit     = "h3"
minute   = 30

# Import data
public   = gpd.read_file(f"../data/2-coverage/healthcare/{unit}/public/lac-driving-{minute}-min.geojson")
official = gpd.read_file(f"../data/2-coverage/healthcare/{unit}/official/lac-driving-{minute}-min.geojson")

# Rename variables 
# Note: rename variables for Atlas IADB (max elength 11 letters)
names    = {"pop_cov":"ncov","pop_uncov":"nuncov","per_cov":"ratecov","per_uncov":"rateunc"}
public   = public  .rename(columns = names)
official = official.rename(columns = names)

In [ ]:
%%time 
if unit == "h3":
    shp0     = get_country_shp(level = 0)   
    public   = gpd.sjoin(public  , shp0)
    official = gpd.sjoin(official, shp0)
    
    public   = public  .drop(columns = "index_right")
    official = official.drop(columns = "index_right")

In [ ]:
%%time 
# Create new featutes 
    # Public data
public["percov"] = public.ncov   * 100 / public.groupby("ADM0_PCODE").ncov  .transform("sum")
public["perunc"] = public.nuncov * 100 / public.groupby("ADM0_PCODE").nuncov.transform("sum")

    # Official data
official["percov"] = official.ncov   * 100 / official.groupby("ADM0_PCODE").ncov  .transform("sum")
official["perunc"] = official.nuncov * 100 / official.groupby("ADM0_PCODE").nuncov.transform("sum")

# Define columns names
colnames   = public.columns.tolist()
vars_      = ['ncov','nuncov','ratecov','rateunc','percov','perunc']
pub_vars   = [f"{name}_pub" if name in vars_ else name for name in colnames]
off_vars   = [f"{name}_off" if name in vars_ else name for name in colnames]

# Rename variables
public  .columns = pub_vars
official.columns = off_vars

# Drop columns from official dataset
idvar    = "ADM2_PCODE" if unit == "adm2" else "hex_id"
off_vars = [idvar] + [name for name in official.columns if "_off" in name] 
official = official[off_vars]

# Merge 
lac = public.merge(official, on = idvar, how = "left")

In [ ]:
# Add poverty data and adjust names if admin-2
# TODO: not neccesary, temporary, remove in other version
if unit == "adm2":
    # Import poverty data 
    path    = "Data Projects/Official Subnational Poverty Rates/lac-level-2.csv"
    poverty = pd.read_csv(f"s3://{sclbucket}/{path}", encoding='latin-1')
    poverty = poverty[["ADM2_PCODE","POVERTY_RATE_TOT","POVERTY_NUM_TOT","POVERTY_SOURCE"]]
    poverty = poverty.rename(columns = {"POVERTY_RATE_TOT":"pov_rate","POVERTY_NUM_TOT":"pov_num","POVERTY_SOURCE":"pov_source"})
    
    # Merge 
    lac = lac.merge(poverty, on = "ADM2_PCODE", how = "left")

In [ ]:
# Drop duplicates
lac = lac.drop_duplicates()

In [ ]:
%%capture
# Export data (GeoJSON, .shp, .csv)
lac.to_file(f"../data/2-coverage/healthcare/{unit}/lac-driving-{min_}-min.geojson", driver = "GeoJSON")
lac.to_file(f"../data/2-coverage/healthcare/{unit}/lac-driving-{min_}-min.shp")
lac.drop(columns = "geometry").to_csv(f"../data/2-coverage/healthcare/{unit}/lac-driving-{min_}-min.csv", index = False)